In [ ]:
from utils import *

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import pandas as pd

from ortools.sat.python import cp_model
from ortools.sat.python.cp_model import _SumArray

In [ ]:
values = getValues()
techs = getTechs()
tribes = getTribes()

In [ ]:
class GameState:
    turn = -1
    player = ""
    game = ""
    parent_state = None
    techs = {}
    previous_techs = {}
    vars = []
    tribe = ""
    cities = [] # 
    child_states = [] # GameStates
    raw_score = None
    full_score = None
    model = None
    solutions = None

    def __init__(self, turn, parent_state=None, previous_tech={}):
        self.turn = turn
        self.model = cp_model.CpModel()
        self.parent_state = parent_sate
        self.previous_tech = previous_tech

    def computeEndScore(self):
        self.

    def __repr__(self):
        return "GameState"
    
    def __str__(self):
        return "GameState turn %d, player %s" % (self.turn, self.player, self.game)

class Player:
    
    initialGameState = []

    def addScore(self):
        if len(initialGameState):
            pass
        pass

    def addTech(self):
        pass

    def addTribe(self, tribe):
        data = getTribe()[tribe]
        pass

In [ ]:
g = GameState()


In [ ]:
def solve(raw_score, full_score):
    model = cp_model.CpModel()
    eq_dict, sym_dict = addAllVars(model, values)

    buildScoreConstraint(model, eq_dict, full_score, True)
    buildScoreConstraint(model, eq_dict, raw_score, False)
    model.Add(_SumArray([sym_dict[e] for e in values['units'].keys()]) < 20)


    solver = cp_model.CpSolver()
    solution_printer = DataFrameSolutionAggregator(sym_dict.values())
    status = solver.SearchForAllSolutions(model, solution_printer)
    return solution_printer

In [ ]:
# superb_ruin
scores = [
    ([515, 515], []),
    ([565, 565], [440, 440]),
    ([590, 600], [450, 450]),
    ([855, 635], [700, 460]),
    ([1385, 670], [1230, 460]),
    ([1860, 1215], [1700, 990]),
    ([2230, 1815], [2010, 1530])
]

In [ ]:
player_scores = [
    [565, 440],
    [590, 450],
    [855, 700],
    [1385, 1230],
    [1860, 1700],
    [2230, 2010]
]

In [ ]:
player_2_scores = [
    [565, 440],
    [600, 450],
    [635, 460],
    [670, 460],
    [1215, 990],
    [1815, 1530]
]

In [ ]:
def divide_scores(scores):


    return selected_scores + divide_score(scores[1:])

In [ ]:
for i in range(len(player_scores) - 1):
    raw_delta = player_scores[i + 1][1] - player_scores[i][1]
    full_delta = player_scores[i + 1][0] - player_scores[i][0]
    print(full_delta, raw_delta)
    # sp = solve(raw_score=raw_delta, full_score=full_delta)
    # print(sp.get_data_frame().to_string())

In [ ]:
print(sp.get_data_frame())

# Equations: 

full score = score(units) + score(tech) + score(city) + score(vision)

raw score =  score(units) + score(tech) + score(city)

'remaining stars + spt + extra stars = stars(units) + stars(tech) + star(building)'

In [1]:
from utils import * 
model = cp_model.CpModel()

past_turn = None
dictionnaries = {}
scores = {
    "start": [515, 390],
    "t0": [565, 440],
    "t1": [600, 450],
    "t2": [635, 460],
    "t3": [670, 460],
    "t4": [1215, 990],
    "t5": [1815, 1530]
}

for turn in ["start", "t0"]:#, "t1", "t2"]:
    dictionnaries[turn] = buildDictionnaries(model, turn)
    full_score = scores[turn][0] 
    raw_score = scores[turn][1]

    buildAllConstraints(model, turn, dictionnaries)

    setRawScore(model, turn, dictionnaries[turn]['scores'], raw_score)
    #setDeltaRawScore(model, score_dict, delta_raw_score)
    setFullScore(model, turn, dictionnaries[turn]['scores'], full_score)
    #setDeltaFullScore(model, score_dict, delta_full_score)
    setTribe(model, turn, "Bardur", dictionnaries[turn]['tribes'])

    linkTurns(model, turn, dictionnaries)
    addInitialState(model, turn, dictionnaries)
    past_turn = turn

model.Add(dictionnaries["start"]['symbols']["start_monument"] == 0)
# model.Add(dictionnaries["start"]['city']["start_level-2_count"] == 0)
# model.Add(dictionnaries["start"]['city']["start_level-5_count"] == 0)
model.Add(dictionnaries["t0"]['symbols']["t0_level-1"] == 1)
model.Add(dictionnaries["t0"]['symbols']["t0_level-2"] == 1)
model.Add(dictionnaries["t0"]['symbols']["t0_level-3"] == 0)
# model.Add(dictionnaries["t0"]['city']["t0_level-1_count"] == 0)
# model.Add(dictionnaries["t0"]['city']["t0_level-2_count"] == 1)
# model.Add(dictionnaries["t0"]['city']["t0_level-3_count"] == 0)
# model.Add(dictionnaries["t0"]['city']["t0_level-4_count"] == 0)
# model.Add(dictionnaries["t0"]['city']["t0_level-5_count"] == 0)

model.Add(dictionnaries["t0"]['symbols']["t0_population"] == 2)
model.Add(dictionnaries["t0"]['symbols']["t0_claimed"] == 9)
model.Add(dictionnaries["t0"]['symbols']["t0_tier-1"] == 1)
model.Add(dictionnaries["t0"]['symbols']["t0_tier-2"] == 0)
model.Add(dictionnaries["t0"]['symbols']["t0_monument"] == 0)
model.Add(dictionnaries["t0"]['symbols']["t0_giant"] == 0)
model.Add(dictionnaries["t0"]['symbols']["t0_warrior"] == 1)
model.Add(dictionnaries["t0"]['symbols']["t0_explorer"] == 0)



solver = cp_model.CpSolver()
output_variables = list([h for b, j in dictionnaries.items() for a,i in j.items() for h in i.values() if a != "equations"])
solution_printer = DataFrameSolutionAggregator(output_variables)
status = solver.SearchForAllSolutions(model, solution_printer)
print('Solve status: %s' % solver.StatusName(status))

((((((10 * start_warrior) + (15 * start_rider) + (15 * start_archer) + (15 * start_shield) + (40 * start_cata) + (25 * start_mindbender) + (25 * start_swordpeople) + (40 * start_knight) + (10 * start_giant))) + ((100 * start_level-1) + (40 * start_level-2) + (35 * start_level-3) + (30 * start_level-4) + (25 * start_level-5) + (5 * start_population) + (400 * start_monument))) + ((100 * start_tier-1) + (200 * start_tier-2) + (300 * start_tier-3))) + ((20 * start_claimed) + (5 * start_revealed) + (150 * start_explorer)))
(start_level-1 + -start_level-2)
((((((10 * t0_warrior) + (15 * t0_rider) + (15 * t0_archer) + (15 * t0_shield) + (40 * t0_cata) + (25 * t0_mindbender) + (25 * t0_swordpeople) + (40 * t0_knight) + (10 * t0_giant))) + ((100 * t0_level-1) + (40 * t0_level-2) + (35 * t0_level-3) + (30 * t0_level-4) + (25 * t0_level-5) + (5 * t0_population) + (400 * t0_monument))) + ((100 * t0_tier-1) + (200 * t0_tier-2) + (300 * t0_tier-3))) + ((20 * t0_claimed) + (5 * t0_revealed) + (150 * 

(((((10 * start_warrior + (15 * start_rider) + (15 * start_archer) + (15 * start_shield) + (40 * start_cata) + (25 * start_mindbender) + (25 * start_swordpeople) + (40 * start_knight) + (10 * start_giant))) + ((100 * start_level-1) + (40 * start_level-2) + (35 * start_level-3) + (30 * start_level-4) + (25 * start_level-5) + (5 * start_population) + (400 * start_monument))) + ((100 * start_tier-1) + (200 * start_tier-2) + (300 * start_tier-3))) + ((20 * start_claimed) + (5 * start_revealed) + (150 * start_explorer)))
((((((10 * t0_warrior) + (15 * t0_rider) + (15 * t0_archer) + (15 * t0_shield) + (40 * t0_cata) + (25 * t0_mindbender) + (25 * t0_swordpeople) + (40 * t0_knight) + (10 * t0_giant))) + ((100 * t0_level-1) + (40 * t0_level-2) + (35 * t0_level-3) + (30 * t0_level-4) + (25 * t0_level-5) + (5 * t0_population) + (400 * t0_monument))) + ((100 * t0_tier-1) + (200 * t0_tier-2) + (300 * t0_tier-3))) + ((20 * t0_claimed) + (5 * t0_revealed) + (150 * t0_explorer)))

In [ ]:
x = solution_printer.get_data_frame()
x

In [ ]:
count = 1
for i in range(len(x.columns)):
    c = list(x.columns)[i]
    print(c, len(x[c].unique()), x[c].unique())
    count *= len(x[c].unique())
count

In [ ]:
x.loc[0]

In [ ]:
x[(x["t0_level-2_spt"] == 1)]# & (x["t0_level-2"] == 1)]

In [ ]:
x[(x["t0_warrior"] == 1)]

In [ ]:
x.loc[8]

In [ ]:
x[(x["t0_monument"] == 0) & (x["start_monument"] == 0)]

In [ ]:
x.drop_duplicates()

9*20 (territories)+ 100 (level 1 city)+ tech+ units= raw score turn 0

turn 0, vision score is always 125=25*5

In [ ]:
score_dict

In [ ]:
"level-6" : 20,
		"level-7" : 15,
		"level-8" : 10,
		"level-9" : 5,
		"level-10" : 0,

1: min = 0, max = 1
2: min = 2, max = 4
3: min = 5, max = 8
4: min = 9, max = 13
5: min = 14, max = 19

b:     min(1)     = 0

x:     min(x)     = max(x - 1) + 1
                  = min(x - 1) + x, 
       max(x)     = min(x) + x

x + 1: min(x + 1) = max(x) + 1
                  = min(x) + x + 1, 
       max(x + 1) = min(x + 1) + x + 1 
                  = min(x) + 2x + 2
x + 2: min(x + 2) = min(x) + 2x + 3 
       max(x + 2) = min(x + 2) + x + 2 
                  = min(x) + 2x + 3  + x + 2 
                  = min(x) + 3x + 5
x + 3: min(x + 3) = min(x + 1) + 2(x + 1) + 3 
                  = min(x) + 3x + 6, 
       max(x + 3) = min(x + 3) + x + 3 
                  = max(x) + x + 9

In [ ]:
Bière 058 469 96 05